In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# 量子近似最適化アルゴリズム

*使用量の目安: Heron r3プロセッサで22分（注意：これは推定値です。実際の実行時間は異なる場合があります。）*
## 背景
このチュートリアルでは、Qiskitパターンの文脈の中で**量子近似最適化アルゴリズム（QAOA）**（量子-古典ハイブリッド反復法）を実装する方法を示します。まず小さなグラフに対する**最大カット**（**Max-Cut**）問題を解き、その後、ユーティリティスケールでの実行方法を学びます。このチュートリアルのすべてのハードウェア実行は、無料でアクセス可能なオープンプランの時間制限内で実行できます。

Max-Cut問題は、解くことが困難な最適化問題（より正確には*NP困難*問題）であり、クラスタリング、ネットワーク科学、統計物理学など、さまざまな応用があります。このチュートリアルでは、辺で接続されたノードからなるグラフを考え、カットによって横断される辺の数が最大化されるようにノードを2つの集合に分割することを目指します。

![Max-Cut問題の図解](../docs/images/tutorials/quantum-approximate-optimization-algorithm/maxcut-illustration.avif)
## 要件
このチュートリアルを開始する前に、以下がインストールされていることを確認してください：
- Qiskit SDK v1.0以降（[可視化](https://docs.quantum.ibm.com/api/qiskit/visualization)サポート付き）
- Qiskit Runtime v0.22以降（`pip install qiskit-ibm-runtime`）

また、[IBM Quantum Platform](/guides/cloud-setup)上のインスタンスへのアクセスが必要です。このチュートリアルはオープンプランでは実行できません。[セッション](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/session)を使用するワークロードを実行するため、プレミアムプランのアクセスが必要です。
## セットアップ

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import rustworkx as rx
from rustworkx.visualization import mpl_draw as draw_graph
import numpy as np
from scipy.optimize import minimize
from collections import defaultdict
from typing import Sequence


from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler

## パート I. 小規模QAOA
チュートリアルの最初のパートでは、小規模なMax-Cut問題を使用して、量子コンピュータで最適化問題を解くためのステップを説明します。

この問題を量子アルゴリズムにマッピングする前に文脈を示すために、Max-Cut問題がどのようにして古典的な組合せ最適化問題になるかをまず理解しましょう。関数 $f(x)$ の最小化を考えます。

$$
\min_{x\in {0, 1}^n}f(x),
$$

ここで、入力 $x$ はグラフの各ノードに対応する成分を持つベクトルです。そして、各成分を $0$ または $1$（カットに含まれるか含まれないかを表す）に制約します。この小規模な例では、$n=5$ ノードのグラフを使用します。

ノードの組 $i,j$ に対して、対応する辺 $(i,j)$ がカットに含まれるかどうかを示す関数を書くことができます。例えば、関数 $x_i + x_j - 2 x_i x_j$ は $x_i$ または $x_j$ のいずれか一方のみが1の場合（つまり辺がカットに含まれる場合）にのみ1となり、それ以外では0になります。カットに含まれる辺を最大化する問題は、次のように定式化できます。

$$
\max_{x\in {0, 1}^n} \sum_{(i,j)} x_i + x_j - 2 x_i x_j,
$$

これは次の形式の最小化問題として書き換えることができます。

$$
\min_{x\in {0, 1}^n} \sum_{(i,j)}  2 x_i x_j - x_i - x_j.
$$

この場合の $f(x)$ の最小値は、カットによって横断される辺の数が最大のときです。ご覧のとおり、ここまでは量子コンピューティングに関連するものはまだありません。この問題を量子コンピュータが理解できる形に再定式化する必要があります。
$n=5$ ノードのグラフを作成して問題を初期化しましょう。

In [2]:
n = 5

graph = rx.PyGraph()
graph.add_nodes_from(np.arange(0, n, 1))
edge_list = [
    (0, 1, 1.0),
    (0, 2, 1.0),
    (0, 4, 1.0),
    (1, 2, 1.0),
    (2, 3, 1.0),
    (3, 4, 1.0),
]
graph.add_edges_from(edge_list)
draw_graph(graph, node_size=600, with_labels=True)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/6ced6bea-0.avif" alt="Output of the previous code cell" />

![前のコードセルの出力](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/6ced6bea-0.avif)

### ステップ1：古典的な入力を量子問題にマッピングする
パターンの最初のステップは、古典的な問題（グラフ）を量子**回路**と**演算子**にマッピングすることです。これを行うには、3つの主要なステップがあります：

1. 一連の数学的再定式化を利用して、この問題を二次制約なし二値最適化（QUBO）問題の記法で表現します。
2. 最適化問題を、基底状態がコスト関数を最小化する解に対応するハミルトニアンとして書き換えます。
3. 量子アニーリングに類似したプロセスを通じて、このハミルトニアンの基底状態を準備する量子回路を作成します。

**注意：** QAOAの方法論では、最終的にハイブリッドアルゴリズムの**コスト関数**を表す演算子（**ハミルトニアン**）と、問題の候補解を表す量子状態を表現するパラメータ化された回路（**アンザッツ**）が必要です。これらの候補状態からサンプリングし、コスト関数を使用して評価することができます。

#### グラフ &rarr; 最適化問題
マッピングの最初のステップは記法の変更です。以下はQUBO記法で問題を表現したものです：

$$
\min_{x\in {0, 1}^n}x^T Q x,
$$

ここで、$Q$ は $n\times n$ の実数行列、$n$ はグラフのノード数、$x$ は上で導入した二値変数のベクトル、$x^T$ はベクトル $x$ の転置を示します。

```
Maximize
 -2*x_0*x_1 - 2*x_0*x_2 - 2*x_0*x_4 - 2*x_1*x_2 - 2*x_2*x_3 - 2*x_3*x_4 + 3*x_0
 + 2*x_1 + 3*x_2 + 2*x_3 + 2*x_4

Subject to
  No constraints

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4
```
### 最適化問題 &rarr; ハミルトニアン
次に、QUBO問題を**ハミルトニアン**（ここではシステムのエネルギーを表す行列）として再定式化できます：

$$
H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
$$

<details>
<summary>
**QAOA問題からハミルトニアンへの再定式化のステップ**
</summary>

QAOA問題がこのように書き換えられることを示すために、まず二値変数 $x_i$ を新しい変数の組 $z_i\in{-1, 1}$ に置き換えます。

$$
x_i = \frac{1-z_i}{2}.
$$

ここで、$x_i$ が $0$ の場合、$z_i$ は $1$ でなければならないことがわかります。最適化問題（$x^TQx$）において $x_i$ を $z_i$ に置き換えると、等価な定式化が得られます。

$$
x^TQx=\sum_{ij}Q_{ij}x_ix_j \\ =\frac{1}{4}\sum_{ij}Q_{ij}(1-z_i)(1-z_j) \\=\frac{1}{4}\sum_{ij}Q_{ij}z_iz_j-\frac{1}{4}\sum_{ij}(Q_{ij}+Q_{ji})z_i + \frac{n^2}{4}.
$$

ここで $b_i=-\sum_{j}(Q_{ij}+Q_{ji})$ と定義し、前の係数と定数項 $n^2$ を除去すると、同じ最適化問題の2つの等価な定式化が得られます。

$$
\min_{x\in{0,1}^n} x^TQx\Longleftrightarrow \min_{z\in{-1,1}^n}z^TQz + b^Tz
$$

ここで、$b$ は $Q$ に依存します。$z^TQz + b^Tz$ を得るために、1/4の係数と最適化に影響しない定数オフセット $n^2$ を省略したことに注意してください。

次に、問題の量子的な定式化を得るために、$z_i$ 変数をパウリ $Z$ 行列（次のような $2\times 2$ 行列）に昇格させます。

$$
Z_i = \begin{pmatrix}1 & 0 \\ 0 & -1\end{pmatrix}.
$$

これらの行列を上記の最適化問題に代入すると、次のハミルトニアンが得られます。

$$
H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
$$

*また、$Z$ 行列は量子コンピュータの計算空間、すなわちサイズ $2^n\times 2^n$ のヒルベルト空間に埋め込まれていることを思い出してください。したがって、$Z_iZ_j$ のような項はテンソル積 $Z_i\otimes Z_j$ として $2^n\times 2^n$ のヒルベルト空間に埋め込まれたものと理解すべきです。例えば、5つの決定変数を持つ問題では、項 $Z_1Z_3$ は $I\otimes Z_3\otimes I\otimes Z_1\otimes I$（$I$ は $2\times 2$ の単位行列）を意味するものと理解されます。*
</details>

このハミルトニアンは**コスト関数ハミルトニアン**と呼ばれます。その基底状態が**コスト関数 $f(x)$ を最小化する**解に対応するという性質を持っています。
したがって、最適化問題を解くには、量子コンピュータ上で $H_C$ の基底状態（またはそれと高い重なりを持つ状態）を準備する必要があります。そして、この状態からサンプリングすることで、高い確率で $min~f(x)$ の解が得られます。
次に、**Max-Cut**問題のハミルトニアン $H_C$ を考えましょう。グラフの各頂点を状態 $|0\rangle$ または $|1\rangle$ の量子ビットに関連付けます。ここで値は頂点が属する集合を表します。問題の目標は、$v_1 = |0\rangle$ かつ $v_2 = |1\rangle$、またはその逆である辺 $(v_1, v_2)$ の数を最大化することです。各量子ビットに $Z$ 演算子を関連付けると、

$$
    Z|0\rangle = |0\rangle \qquad Z|1\rangle = -|1\rangle
$$

辺 $(v_1, v_2)$ がカットに属するのは、$(Z_1|v_1\rangle) \cdot (Z_2|v_2\rangle)$ の固有値が $-1$ の場合、つまり $v_1$ と $v_2$ に関連付けられた量子ビットが異なる場合です。同様に、$(Z_1|v_1\rangle) \cdot (Z_2|v_2\rangle)$ の固有値が $1$ の場合、$(v_1, v_2)$ はカットに属しません。各頂点に関連付けられた正確な量子ビットの状態には関心がなく、辺を挟んで同じかどうかのみに関心があることに注意してください。Max-Cut問題では、以下のハミルトニアンの固有値が最小化されるような頂点上の量子ビットの割り当てを見つける必要があります。
$$
    H_C = \sum_{(i,j) \in e} Q_{ij} \cdot Z_i Z_j.
$$

言い換えると、Max-Cut問題ではすべての $i$ に対して $b_i = 0$ です。$Q_{ij}$ の値は辺の重みを表します。このチュートリアルでは重みなしグラフを考えます。つまり、すべての $i, j$ に対して $Q_{ij} = 1.0$ です。

In [ ]:
def build_max_cut_paulis(
    graph: rx.PyGraph,
) -> list[tuple[str, list[int], float]]:
    """Convert the graph to Pauli list.

    This function does the inverse of `build_max_cut_graph`
    """
    pauli_list = []
    for edge in list(graph.edge_list()):
        weight = graph.get_edge_data(edge[0], edge[1])
        pauli_list.append(("ZZ", [edge[0], edge[1]], weight))
    return pauli_list


max_cut_paulis = build_max_cut_paulis(graph)
cost_hamiltonian = SparsePauliOp.from_sparse_list(max_cut_paulis, n)
print("Cost Function Hamiltonian:", cost_hamiltonian)

Cost Function Hamiltonian: SparsePauliOp(['IIIZZ', 'IIZIZ', 'ZIIIZ', 'IIZZI', 'IZZII', 'ZZIII'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])


#### Hamiltonian &rarr; quantum circuit

The Hamiltonian $H_C$ contains the quantum definition of your problem. Now you can create a quantum circuit that will help *sample* good solutions from the quantum computer. The QAOA is inspired by quantum annealing and applies alternating layers of operators in the quantum circuit.

The general idea is to start in the ground state of a known system, $H^{\otimes n}|0\rangle$ above, and then steer the system into the ground state of the cost operator that you are interested in. This is done by applying the operators $\exp\{-i\gamma_k H_C\}$ and $\exp\{-i\beta_k H_m\}$ with angles $\gamma_1,...,\gamma_p$ and $\beta_1,...,\beta_p~$.


The quantum circuit that you generate is **parametrized** by $\gamma_i$ and $\beta_i$, so you can try out different values of $\gamma_i$ and $\beta_i$ and sample from the resulting state.

![Circuit diagram with QAOA layers](../docs/images/tutorials/quantum-approximate-optimization-algorithm/circuit-diagram.svg)


In this case, you will try an example with one QAOA layer that contains two parameters: $\gamma_1$ and $\beta_1$.

In [4]:
circuit = QAOAAnsatz(cost_operator=cost_hamiltonian, reps=2)
circuit.measure_all()

circuit.draw("mpl")

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/7bd8c6d4-f40f-4a11-a440-0b26d9021b53-0.avif" alt="Output of the previous code cell" />

In [5]:
circuit.parameters

ParameterView([ParameterVectorElement(β[0]), ParameterVectorElement(β[1]), ParameterVectorElement(γ[0]), ParameterVectorElement(γ[1])])

### Step 2: Optimize problem for quantum hardware execution

The circuit above contains a series of abstractions useful to think about quantum algorithms, but not possible to run on the hardware. To be able to run on a QPU, the circuit needs to undergo a series of operations that make up the **transpilation** or **circuit optimization** step of the pattern.

The Qiskit library offers a series of **transpilation passes** that cater to a wide range of circuit transformations. You need to make sure that your circuit is **optimized** for your purpose.

Transpilation may involves several steps, such as:

* **Initial mapping** of the qubits in the circuit (such as decision variables) to physical qubits on the device.
* **Unrolling** of the instructions in the quantum circuit to the hardware-native instructions that the backend understands.
* **Routing** of any qubits in the circuit that interact to physical qubits that are adjacent with one another.
* **Error suppression** by adding single-qubit gates to suppress noise with dynamical decoupling.


More information about transpilation is available in our [documentation](/docs/guides/transpile).

The following code transforms and optimizes the abstract circuit into a format that is ready for execution on one of devices accessible through the cloud using the **Qiskit IBM Runtime service**.

In [6]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
print(backend)

# Create pass manager for transpilation
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

candidate_circuit = pm.run(circuit)
candidate_circuit.draw("mpl", fold=False, idle_wires=False)

<IBMBackend('test_heron_pok_1')>


<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3f28a422-805c-4d3d-b5f6-62539e9133bd-1.avif" alt="Output of the previous code cell" />

### Step 3: Execute using Qiskit primitives

In the QAOA workflow, the optimal QAOA parameters are found in an iterative optimization loop, which runs a series of circuit evaluations and uses a classical optimizer to find the optimal $\beta_k$ and $\gamma_k$ parameters. This execution loop is executed via the following steps:

1. Define the initial parameters
2. Instantiate a new `Session` containing the optimization loop and the primitive used to sample the circuit
3. Once an optimal set of parameters is found, execute the circuit a final time to obtain a final distribution which will be used in the post-process step.

#### Define circuit with initial parameters
We start with arbitrary chosen parameters.

In [7]:
initial_gamma = np.pi
initial_beta = np.pi / 2
init_params = [initial_beta, initial_beta, initial_gamma, initial_gamma]

### ステップ2：量子ハードウェア実行のための問題の最適化
上記の回路には量子アルゴリズムを考える上で有用な一連の抽象化が含まれていますが、ハードウェア上でそのまま実行することはできません。QPU上で実行するには、パターンの**トランスパイル**または**回路最適化**ステップを構成する一連の操作を回路に施す必要があります。

Qiskitライブラリは、幅広い回路変換に対応する一連の**トランスパイルパス**を提供しています。回路が目的に合わせて**最適化**されていることを確認する必要があります。

トランスパイルには、以下のようないくつかのステップが含まれる場合があります：

* 回路内の量子ビット（決定変数など）のデバイス上の物理量子ビットへの**初期マッピング**。
* 量子回路内の命令を、バックエンドが理解できるハードウェアネイティブ命令への**展開**。
* 相互作用する回路内の量子ビットを、隣接する物理量子ビットへの**ルーティング**。
* 動的デカップリングによる単一量子ビットゲートの追加による**エラー抑制**。

トランスパイルの詳細については、[ドキュメント](/guides/transpile)をご覧ください。

以下のコードは、抽象回路を変換・最適化し、**Qiskit IBM Runtimeサービス**を通じてアクセス可能なデバイスの1つで実行できる形式にします。

In [8]:
def cost_func_estimator(params, ansatz, hamiltonian, estimator):
    # transform the observable defined on virtual qubits to
    # an observable defined on all physical qubits
    isa_hamiltonian = hamiltonian.apply_layout(ansatz.layout)

    pub = (ansatz, isa_hamiltonian, params)
    job = estimator.run([pub])

    results = job.result()[0]
    cost = results.data.evs

    objective_func_vals.append(cost)

    return cost

In [9]:
objective_func_vals = []  # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)
    estimator.options.default_shots = 1000

    # Set simple error suppression/mitigation options
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XY4"
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit, cost_hamiltonian, estimator),
        method="COBYLA",
        tol=1e-2,
    )
    print(result)

 message: Return from COBYLA because the trust region radius reaches its lower bound.
 success: True
  status: 0
     fun: -1.6295230263157894
       x: [ 1.530e+00  1.439e+00  4.071e+00  4.434e+00]
    nfev: 26
   maxcv: 0.0


![前のコードセルの出力](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3f28a422-805c-4d3d-b5f6-62539e9133bd-1.avif)

### ステップ3：Qiskitプリミティブを使用した実行
QAOAワークフローでは、最適なQAOAパラメータは反復的な最適化ループで見つけられます。このループは一連の回路評価を実行し、古典オプティマイザを使用して最適な $\beta_k$ と $\gamma_k$ パラメータを見つけます。この実行ループは以下のステップで実行されます：

1. 初期パラメータを定義します。
2. 最適化ループと回路をサンプリングするプリミティブを含む新しい`Session`をインスタンス化します。
3. 最適なパラメータのセットが見つかったら、最終的な分布を得るために回路を最後にもう一度実行します。この分布は後処理ステップで使用されます。
#### 初期パラメータによる回路の定義
任意に選択したパラメータから開始します。

In [10]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/e14ecc92-0.avif" alt="Output of the previous code cell" />

#### バックエンドと実行プリミティブの定義
IBM&reg; バックエンドとの対話には**Qiskit Runtimeプリミティブ**を使用します。2つのプリミティブはSamplerとEstimatorであり、プリミティブの選択は量子コンピュータ上で実行したい測定の種類によって異なります。$H_C$ の最小化には、コスト関数の測定が単に期待値 $\langle H_C \rangle$ であるため、Estimatorを使用します。
#### 実行
プリミティブは量子デバイス上でワークロードをスケジュールするためのさまざまな[実行モード](/guides/execution-modes)を提供しており、QAOAワークフローはセッション内で反復的に実行されます。

![シングルジョブ、バッチ、セッションの各ランタイムモードの動作を示す図解。](../docs/images/tutorials/quantum-approximate-optimization-algorithm/runtime-modes.avif)

サンプラーベースのコスト関数をSciPyの最小化ルーチンに組み込んで、最適なパラメータを見つけることができます。

In [11]:
optimized_circuit = candidate_circuit.assign_parameters(result.x)
optimized_circuit.draw("mpl", fold=False, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/2989e76e-4296-4dd8-b065-2b8fced064cf-0.avif" alt="Output of the previous code cell" />

In [12]:
# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"

pub = (optimized_circuit,)
job = sampler.run([pub], shots=int(1e4))
counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_int = {key: val / shots for key, val in counts_int.items()}
final_distribution_bin = {key: val / shots for key, val in counts_bin.items()}
print(final_distribution_int)

{28: 0.0328, 11: 0.0343, 2: 0.0296, 25: 0.0308, 16: 0.0303, 27: 0.0302, 13: 0.0323, 7: 0.0312, 4: 0.0296, 9: 0.0295, 26: 0.0321, 30: 0.031, 23: 0.0324, 31: 0.0303, 21: 0.0335, 15: 0.0317, 12: 0.0309, 29: 0.0297, 3: 0.0313, 5: 0.0312, 6: 0.0274, 10: 0.0329, 22: 0.0353, 0: 0.0315, 20: 0.0326, 8: 0.0322, 14: 0.0306, 17: 0.0295, 18: 0.0279, 1: 0.0325, 24: 0.0334, 19: 0.0295}


### Step 4: Post-process and return result in desired classical format

The post-processing step interprets the sampling output to return a solution for your original problem. In this case, you are interested in the bitstring with the highest probability as this determines the optimal cut. The symmetries in the problem allow for four possible solutions, and the sampling process will return one of them with a slightly higher probability, but you can see in the plotted distribution below that four of the bitstrings are distinctively more likely than the rest.

In [13]:
# auxiliary functions to sample most likely bitstring
def to_bitstring(integer, num_bits):
    result = np.binary_repr(integer, width=num_bits)
    return [int(digit) for digit in result]


keys = list(final_distribution_int.keys())
values = list(final_distribution_int.values())
most_likely = keys[np.argmax(np.abs(values))]
most_likely_bitstring = to_bitstring(most_likely, len(graph))
most_likely_bitstring.reverse()

print("Result bitstring:", most_likely_bitstring)

Result bitstring: [0, 1, 1, 0, 1]


In [14]:
matplotlib.rcParams.update({"font.size": 10})
final_bits = final_distribution_bin
values = np.abs(list(final_bits.values()))
top_4_values = sorted(values, reverse=True)[:4]
positions = []
for value in top_4_values:
    positions.append(np.where(values == value)[0])
fig = plt.figure(figsize=(11, 6))
ax = fig.add_subplot(1, 1, 1)
plt.xticks(rotation=45)
plt.title("Result Distribution")
plt.xlabel("Bitstrings (reversed)")
plt.ylabel("Probability")
ax.bar(list(final_bits.keys()), list(final_bits.values()), color="tab:grey")
for p in positions:
    ax.get_children()[int(p[0])].set_color("tab:purple")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/650875e9-adbc-43bd-9505-556be2566278-0.avif" alt="Output of the previous code cell" />

![前のコードセルの出力](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/e14ecc92-0.avif)

回路の最適なパラメータが見つかったら、これらのパラメータを割り当て、最適化されたパラメータで得られた最終的な分布をサンプリングできます。ここでは*Sampler*プリミティブを使用すべきです。ビット列測定の確率分布がグラフの最適カットに対応するためです。

**注意：** これは量子コンピュータ内で量子状態 $\psi$ を準備し、それを測定することを意味します。測定により状態は単一の計算基底状態（例えば `010101110000...`）に崩壊します。これは初期の最適化問題（タスクに応じて $\max f(x)$ または $\min f(x)$）の候補解 $x$ に対応します。

In [15]:
# auxiliary function to plot graphs
def plot_result(G, x):
    colors = ["tab:grey" if i == 0 else "tab:purple" for i in x]
    pos, _default_axes = rx.spring_layout(G), plt.axes(frameon=True)
    rx.visualization.mpl_draw(
        G, node_color=colors, node_size=100, alpha=0.8, pos=pos
    )


plot_result(graph, most_likely_bitstring)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/33135970-8bc4-4fb2-ab87-08726a432ce4-0.avif" alt="Output of the previous code cell" />

And calculate the value of the cut:

In [16]:
def evaluate_sample(x: Sequence[int], graph: rx.PyGraph) -> float:
    assert len(x) == len(
        list(graph.nodes())
    ), "The length of x must coincide with the number of nodes in the graph."
    return sum(
        x[u] * (1 - x[v]) + x[v] * (1 - x[u])
        for u, v in list(graph.edge_list())
    )


cut_value = evaluate_sample(most_likely_bitstring, graph)
print("The value of the cut is:", cut_value)

The value of the cut is: 5


## Part II. Scale it up!

You have access to many devices with over 100 qubits on IBM Quantum&reg; Platform. Select one on which to solve Max-Cut on a 100-node weighted graph. This is a "utility-scale" problem. The steps to build the workflow are followed the same as above, but with a much larger graph.

In [17]:
n = 100  # Number of nodes in graph
graph_100 = rx.PyGraph()
graph_100.add_nodes_from(np.arange(0, n, 1))
elist = []
for edge in backend.coupling_map:
    if edge[0] < n and edge[1] < n:
        elist.append((edge[0], edge[1], 1.0))
graph_100.add_edges_from(elist)
draw_graph(graph_100, node_size=200, with_labels=True, width=1)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/590fe2ce-0.avif" alt="Output of the previous code cell" />

### ステップ4：後処理と望ましい古典形式での結果の返却
後処理ステップでは、サンプリング出力を解釈して元の問題の解を返します。この場合、最も高い確率を持つビット列が最適カットを決定するため、それに関心があります。問題の対称性により4つの可能な解があり、サンプリングプロセスはそのうちの1つをわずかに高い確率で返しますが、以下のプロットされた分布では4つのビット列が他のものよりも明確に高い確率を持っていることがわかります。

In [18]:
max_cut_paulis_100 = build_max_cut_paulis(graph_100)

cost_hamiltonian_100 = SparsePauliOp.from_sparse_list(max_cut_paulis_100, 100)
print("Cost Function Hamiltonian:", cost_hamiltonian_100)

Cost Function Hamiltonian: SparsePauliOp(['...'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0

#### Hamiltonian &rarr; quantum circuit

In [19]:
circuit_100 = QAOAAnsatz(cost_operator=cost_hamiltonian_100, reps=1)
circuit_100.measure_all()

circuit_100.draw("mpl", fold=False, scale=0.2, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/9693adfc-0.avif" alt="Output of the previous code cell" />

### Step 2: Optimize problem for quantum execution
To scale the circuit optimization step to utility-scale problems, you can take advantage of the high performance transpilation strategies introduced in Qiskit SDK v1.0. Other tools include the new transpiler service with [AI enhanced transpiler passes](/docs/guides/ai-transpiler-passes).

In [20]:
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

candidate_circuit_100 = pm.run(circuit_100)
candidate_circuit_100.draw("mpl", fold=False, scale=0.1, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3a14e7ad-0.avif" alt="Output of the previous code cell" />

![前のコードセルの出力](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/650875e9-adbc-43bd-9505-556be2566278-0.avif)

#### 最適カットの可視化
最適なビット列から、このカットを元のグラフ上で可視化できます。

In [21]:
initial_gamma = np.pi
initial_beta = np.pi / 2
init_params = [initial_beta, initial_gamma]

objective_func_vals = []  # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)

    estimator.options.default_shots = 1000

    # Set simple error suppression/mitigation options
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XY4"
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit_100, cost_hamiltonian_100, estimator),
        method="COBYLA",
    )
    print(result)

 message: Return from COBYLA because the trust region radius reaches its lower bound.
 success: True
  status: 0
     fun: -3.9939191365979383
       x: [ 1.571e+00  3.142e+00]
    nfev: 29
   maxcv: 0.0


![前のコードセルの出力](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/33135970-8bc4-4fb2-ab87-08726a432ce4-0.avif)

そして、カットの値を計算します：

In [22]:
optimized_circuit_100 = candidate_circuit_100.assign_parameters(result.x)
optimized_circuit_100.draw("mpl", fold=False, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/1c432c2e-0.avif" alt="Output of the previous code cell" />

Finally, execute the circuit with the optimal parameters to sample from the corresponding distribution.

In [23]:
# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"


pub = (optimized_circuit_100,)
job = sampler.run([pub], shots=int(1e4))

counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_100_int = {
    key: val / shots for key, val in counts_int.items()
}

## パート II. スケールアップ！
IBM Quantum&reg; Platform上には、100量子ビット以上のデバイスが多数あります。その中から1つを選び、100ノードの重み付きグラフに対するMax-Cut問題を解きます。これは「ユーティリティスケール」の問題です。ワークフローを構築する手順は上記と同じですが、はるかに大きなグラフを使用します。

In [24]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/0fda3611-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/590fe2ce-0.avif)

### ステップ 1: 古典的な入力を量子問題にマッピングする
#### グラフ &rarr; ハミルトニアン
まず、解きたいグラフをQAOAに適したハミルトニアンに直接変換します。

In [25]:
_PARITY = np.array(
    [-1 if bin(i).count("1") % 2 else 1 for i in range(256)],
    dtype=np.complex128,
)


def evaluate_sparse_pauli(state: int, observable: SparsePauliOp) -> complex:
    """Utility for the evaluation of the expectation value of a measured state."""
    packed_uint8 = np.packbits(observable.paulis.z, axis=1, bitorder="little")
    state_bytes = np.frombuffer(
        state.to_bytes(packed_uint8.shape[1], "little"), dtype=np.uint8
    )
    reduced = np.bitwise_xor.reduce(packed_uint8 & state_bytes, axis=1)
    return np.sum(observable.coeffs * _PARITY[reduced])


def best_solution(samples, hamiltonian):
    """Find solution with lowest cost"""
    min_cost = 1000
    min_sol = None
    for bit_str in samples.keys():
        # Qiskit use little endian hence the [::-1]
        candidate_sol = int(bit_str)
        # fval = qp.objective.evaluate(candidate_sol)
        fval = evaluate_sparse_pauli(candidate_sol, hamiltonian).real
        if fval <= min_cost:
            min_sol = candidate_sol

    return min_sol


best_sol_100 = best_solution(final_distribution_100_int, cost_hamiltonian_100)
best_sol_bitstring_100 = to_bitstring(int(best_sol_100), len(graph_100))
best_sol_bitstring_100.reverse()

print("Result bitstring:", best_sol_bitstring_100)

Result bitstring: [1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1]


Next, visualize the cut. Nodes of the same color belong to the same group.

In [26]:
plot_result(graph_100, best_sol_bitstring_100)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/b4a25e28-0.avif" alt="Output of the previous code cell" />

#### ハミルトニアン &rarr; 量子回路

In [27]:
cut_value_100 = evaluate_sample(best_sol_bitstring_100, graph_100)
print("The value of the cut is:", cut_value_100)

The value of the cut is: 124


![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/9693adfc-0.avif)

### ステップ 2: 量子実行のための問題の最適化
回路の最適化ステップをユーティリティスケールの問題に拡張するために、Qiskit SDK v1.0で導入された高性能トランスパイル戦略を活用できます。その他のツールとして、[AI強化トランスパイラパス](/guides/ai-transpiler-passes)を備えた新しいトランスパイラサービスもあります。

In [28]:
# auxiliary function to help plot cumulative distribution functions
def _plot_cdf(objective_values: dict, ax, color):
    x_vals = sorted(objective_values.keys(), reverse=True)
    y_vals = np.cumsum([objective_values[x] for x in x_vals])
    ax.plot(x_vals, y_vals, color=color)


def plot_cdf(dist, ax, title):
    _plot_cdf(
        dist,
        ax,
        "C1",
    )
    ax.vlines(min(list(dist.keys())), 0, 1, "C1", linestyle="--")

    ax.set_title(title)
    ax.set_xlabel("Objective function value")
    ax.set_ylabel("Cumulative distribution function")
    ax.grid(alpha=0.3)


# auxiliary function to convert bit-strings to objective values
def samples_to_objective_values(samples, hamiltonian):
    """Convert the samples to values of the objective function."""

    objective_values = defaultdict(float)
    for bit_str, prob in samples.items():
        candidate_sol = int(bit_str)
        fval = evaluate_sparse_pauli(candidate_sol, hamiltonian).real
        objective_values[fval] += prob

    return objective_values

In [29]:
result_dist = samples_to_objective_values(
    final_distribution_100_int, cost_hamiltonian_100
)

Finally, you can plot the cumulative distribution function to visualize how each sample contributes to the total probability distribution and the corresponding objective value. The horizontal spread shows the range of objective values of the samples in the final distribution. Ideally, you would see that the cumulative distribution function has "jumps" at the lower end of the objective function value axis. This would mean that few solutions with low cost have high probability of being sampled. A smooth, wide curve indicates that each sample is similarly likely, and they can have very different objective values, low or high.

In [30]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
plot_cdf(result_dist, ax, "Eagle device")

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/4381a2b3-0.avif" alt="Output of the previous code cell" />

デバイス上でQAOAを実行して最適なパラメータが見つかったら、そのパラメータを回路に割り当てます。